# React

Implement a basic reactive system.

Reactive programming is a programming paradigm that focuses on how values
are computed in terms of each other to allow a change to one value to
automatically propagate to other values, like in a spreadsheet.

Implement a basic reactive system with cells with settable values ("input"
cells) and cells with values computed in terms of other cells ("compute"
cells). Implement updates so that when an input value is changed, values
propagate to reach a new stable system state.

In addition, compute cells should allow for registering change notification
callbacks.  Call a cell’s callbacks when the cell’s value in a new stable
state has changed from the previous stable state.

## Exception messages

Sometimes it is necessary to raise an exception. When you do this, you should include a meaningful error message to
indicate what the source of the error is. This makes your code more readable and helps significantly with debugging. Not
every exercise will require you to raise an exception, but for those that do, the tests will only pass if you include
a message.

To raise a message with an exception, just write it as an argument to the exception type. For example, instead of
`raise Exception`, you shold write:

```python
raise Exception("Meaningful message indicating the source of the error")
```

## Submitting Exercises

Note that, when trying to submit an exercise, make sure the solution is in the `$EXERCISM_WORKSPACE/python/react` directory.

You can find your Exercism workspace by running `exercism debug` and looking for the line that starts with `Workspace`.

For more detailed information about running tests, code style and linting,
please see the [help page](http://exercism.io/languages/python).

## Submitting Incomplete Solutions
It's possible to submit an incomplete solution so you can see how others have completed the exercise.


In [ ]:
class Cell(object):
    def set_value(value):
        pass

    def add_watcher(self, watcher_callback):
        pass

    def remove_watcher(self, watcher_callback):
        pass


class Reactor(object):
    def create_input_cell(self, value):
        pass

    def create_compute_cell(self, dependencies, updater_callback):
        pass


In [ ]:
import unittest



class CallbackManager(object):
    counter = 0
    observed1 = []
    observed2 = []

    @staticmethod
    def reset():
        CallbackManager.counter = 0
        CallbackManager.observed1 = []
        CallbackManager.observed2 = []

    @staticmethod
    def count(sender, value):
        CallbackManager.counter += 1

    @staticmethod
    def observe1(sender, value):
        CallbackManager.observed1.append(value)

    @staticmethod
    def observe2(sender, value):
        CallbackManager.observed2.append(value)


def increment(values):
    return values[0] + 1


def decrement(values):
    return values[0] - 1


def product(values):
    return values[0] * values[1]


def minimum_of_2(values):
    return values[0] + 1 if values[0] > 2 else 2


class ReactTest(unittest.TestCase):
    def test_setting_input_changes_value(self):
        reactor = Reactor()
        inputCell1 = reactor.create_input_cell(1)
        self.assertEqual(inputCell1.value, 1)
        inputCell1.set_value(2)
        self.assertEqual(inputCell1.value, 2)

    def test_computed_cell_determined_by_dependencies(self):
        reactor = Reactor()
        inputCell1 = reactor.create_input_cell(1)
        computeCell1 = reactor.create_compute_cell({inputCell1}, increment)

        self.assertEqual(computeCell1.value, 2)
        inputCell1.set_value(2)
        self.assertEqual(computeCell1.value, 3)

    def test_compute_cells_determined_by_other_compute_cells(self):
        reactor = Reactor()
        inputCell1 = reactor.create_input_cell(1)
        computeCell1 = reactor.create_compute_cell({inputCell1}, increment)
        computeCell2 = reactor.create_compute_cell({inputCell1}, decrement)
        computeCell3 = reactor.create_compute_cell({computeCell1,
                                                    computeCell2},
                                                   product)
        self.assertEqual(computeCell3.value, 0)
        inputCell1.set_value(3)
        self.assertEqual(computeCell3.value, 8)

    def test_compute_cells_can_have_callbacks(self):
        reactor = Reactor()
        inputCell1 = reactor.create_input_cell(1)
        computeCell1 = reactor.create_compute_cell({inputCell1}, increment)
        observed = []
        computeCell1.add_watcher(lambda sender, value: observed.append(value))
        self.assertEqual(observed, [])
        inputCell1.set_value(2)
        self.assertEqual(observed, [3])

    def test_callbacks__only_trigger_on_change(self):
        reactor = Reactor()
        inputCell1 = reactor.create_input_cell(1)
        computeCell1 = reactor.create_compute_cell({inputCell1}, minimum_of_2)

        CallbackManager.reset()
        computeCell1.add_watcher(CallbackManager.count)

        inputCell1.set_value(1)
        self.assertEqual(CallbackManager.counter, 0)
        inputCell1.set_value(2)
        self.assertEqual(CallbackManager.counter, 0)
        inputCell1.set_value(3)
        self.assertEqual(CallbackManager.counter, 1)

    def test_callbacks_can_be_removed(self):
        reactor = Reactor()
        inputCell1 = reactor.create_input_cell(1)
        computeCell1 = reactor.create_compute_cell({inputCell1}, increment)

        CallbackManager.reset()
        computeCell1.add_watcher(CallbackManager.observe1)
        computeCell1.add_watcher(CallbackManager.observe2)

        inputCell1.set_value(2)
        self.assertEqual(CallbackManager.observed1, [3])
        self.assertEqual(CallbackManager.observed2, [3])

        computeCell1.remove_watcher(CallbackManager.observe1)
        inputCell1.set_value(3)
        self.assertEqual(CallbackManager.observed1, [3])
        self.assertEqual(CallbackManager.observed2, [3, 4])

    def test_callbacks_only_called_once(self):
        reactor = Reactor()
        inputCell1 = reactor.create_input_cell(1)
        computeCell1 = reactor.create_compute_cell({inputCell1}, increment)
        computeCell2 = reactor.create_compute_cell({inputCell1}, decrement)
        computeCell3 = reactor.create_compute_cell({computeCell2}, decrement)
        computeCell4 = reactor.create_compute_cell({computeCell1,
                                                    computeCell3},
                                                   product)

        CallbackManager.reset()
        computeCell4.add_watcher(CallbackManager.count)

        inputCell1.set_value(3)
        self.assertEqual(CallbackManager.counter, 1)


unittest.main(argv=[''], exit=False)
